In [1]:
#include <iostream>
#include <string>
#include <math.h>       /* cosh, log */
using namespace std;

In [1]:
//Subo el directorio donde estan los headers de la librería de Delphes
gSystem->AddIncludePath("/home/camilo/Proyectos/Delphes/delphes/external/ExRootAnalysis");

In [ ]:
#include

In [2]:
/*Cargo la librería de Delphes*/
gSystem->Load("/home/camilo/Proyectos/Delphes/delphes/libDelphes.so");

Error in <TCling::RegisterModule>: cannot find dictionary module libClassesDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libDisplayDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libFastJetDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libModulesDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libExRootAnalysisDict_rdict.pcm


In [3]:
/*Instancio un objeto TChain*/
TChain chain("Delphes");

In [4]:
/*Creo las variables necesarias para el for*/
//std::string nom1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/Signal_prueba-2/Events/run_";
std::string nom1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/BackGround-DY_2j/Events/run_";
//std::string nom1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/Signal_2j_mu-nu/Events/run_";
//std::string nom1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/BackGround-tW/Events/run_";
//std::string nom1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/BackGround-WW/Events/run_";
std::string nom2="/tag_1_delphes_events.root";
std::string num, out;

In [7]:
/*Llenado del chain con todos los runs*/
for ( i=1; i<3; i=i+1 ) {
   num = std::to_string(i);
   if( i < 10 ) {
       out= nom1+"0"+num+nom2;
    }
//convierto en cadena de caracteres constante (es lo que recibe la función)
    const char * c = out.c_str();
/*pego todos los datos*/
    chain.Add(c);
}

In [8]:
//Creo un objeto de treereader, que recive la dirección en memoria de
//la cadena (TChain) como argumento
ExRootTreeReader *treeReader = new ExRootTreeReader(&chain);

input_line_48:4:19: error: redefinition of 'treeReader'
ExRootTreeReader *treeReader = new ExRootTreeReader(&chain);
                  ^
input_line_45:4:19: note: previous definition is here
ExRootTreeReader *treeReader = new ExRootTreeReader(&chain);
                  ^


In [9]:
/*Leo el numero de entradas*/
Long64_t numberOfEntries = treeReader->GetEntries()

(long long) 240996


In [10]:
/*Obtengo las ramas que deseo*/
TClonesArray *branchMuon = treeReader->UseBranch("Muon");
TClonesArray *branchMissingET = treeReader->UseBranch("MissingET");
TClonesArray *branchJet = treeReader->UseBranch("Jet");

In [11]:
// Inicializo los histogramas
TH1 *histJetPT = new TH1F("jet_pt", "jet P_{T}", 100, 0.0, 100.0);
TH1 *histMuon = new TH1F("mass", "M_{inv}(mu_{1}, mu_{2})", 100, 40.0, 140.0);

In [12]:
// Inicializo los contadores
Int_t cut1=0, cut2=0, cut3=0, cut4=0, cut5=0;

In [13]:
for(Int_t entry = 0; entry < numberOfEntries; ++entry)
    {
        // Se carga un evento especifico para ser analizado
        treeReader->ReadEntry(entry);

        // Se hace el primer corte, se pide que tenga al menos dos jets
        if(branchJet->GetEntries() > 1)
        {
            ++cut1; //Cuantos eventos pasan el corte 1
            
            Int_t count=0;

            for (Int_t i=0; i<branchJet->GetEntries(); ++i)
            {
                Jet* jet = (Jet*) branchJet->At(i);
                if(jet->BTag>0) //BTag 0 si no es b, 1 si sí es b
                {
                    ++count;
                }
    
            }

            if( count>1 )
            {
                ++cut2; //Cuantos eventos pasan el corte 2
                //Inicializo los obejos clase muon
                Muon *muon1, *muon2;

                // Para ser contados deben de tener al menos 2 muones
                if(branchMuon->GetEntries() > 1)
                {
                    ++cut3; //Cuantos eventos pasan el corte 3
                    // Take first two electrons, tomo solo dos porque estan organizados por PT
                    muon1 = (Muon *) branchMuon->At(0);
                    muon2 = (Muon *) branchMuon->At(1);
                    //Si la carga es diferente
                    int charge = muon1->Charge * muon2->Charge;
                    if( charge < 0 )
                    {
                        ++cut4; //Cuantos eventos pasan el corte 4
                
                        double MassMuon=(muon1->P4()+muon2->P4()).M();
                        if( !((MassMuon>80)&&(MassMuon<100)) )
                        {
                            ++cut5; //Cuantos eventos pasan el corte 4
                     
                            
                            // Plot jet transverse momentum
                            //histJetPT->Fill(jet->PT);
                            // Plot their invariant mass
                            //histMass->Fill(((elec1->P4()) + (elec2->P4())).M());
               
                        
                        
                        }
                    }
                }
            }
        }
    }

In [14]:
cout<<"#Raw ="<<numberOfEntries<<endl;
cout<<"#Cut1="<<cut1<<endl;
cout<<"#Cut2="<<cut2<<endl;
cout<<"#Cut3="<<cut3<<endl;
cout<<"#Cut4="<<cut4<<endl;
cout<<"#Cut5="<<cut5<<endl<<endl;


float EFFN1=(float)cut1/(float)numberOfEntries;
float EFFN2=(float)cut2/(float)cut1;
float EFFN3=(float)cut3/(float)cut2;
float EFFN4=(float)cut4/(float)cut3;
float EFFN5=(float)cut5/(float)cut4;


float EFFA1=(float)cut1/(float)numberOfEntries;
float EFFA2=(float)cut2/(float)numberOfEntries;
float EFFA3=(float)cut3/(float)numberOfEntries;
float EFFA4=(float)cut4/(float)numberOfEntries;
float EFFA5=(float)cut5/(float)numberOfEntries;

cout<<"EFFA1="<<EFFA1<<endl;
cout<<"EFFA2="<<EFFA2<<endl;
cout<<"EFFA3="<<EFFA3<<endl;
cout<<"EFFA4="<<EFFA4<<endl;
cout<<"EFFA5="<<EFFA5<<endl<<endl;


cout<<"EFFN1="<<EFFN1<<endl;
cout<<"EFFN2="<<EFFN2<<endl;
cout<<"EFFN3="<<EFFN3<<endl;
cout<<"EFFN4="<<EFFN4<<endl;
cout<<"EFFN5="<<EFFN5<<endl;

#Raw =240996
#Cut1=42795
#Cut2=462
#Cut3=201
#Cut4=201
#Cut5=18

EFFA1=0.177576
EFFA2=0.00191704
EFFA3=0.000834039
EFFA4=0.000834039
EFFA5=7.469e-05

EFFN1=0.177576
EFFN2=0.0107957
EFFN3=0.435065
EFFN4=1
EFFN5=0.0895522


In [ ]:
38407
36208
7244
4037
4037
4037

In [ ]:
MassMuon=std::sqrt(2*(muon1->PT * muon2->PT)*(std::cosh(muon1->Eta-muon2->Eta)-std::cos(muon1->Phi-muon2->Phi)))

In [ ]:
std::vector<Muon> vectorOfMuons;

In [ ]:
//Llenado de vector de muones del evento
std::vector<Muon*> vectorOfMuons;
vectorOfMuons.reserve(branchMuon->GetEntries());
for (int i=0; i<branchMuon->GetEntries(); ++i)
{
    Muon* muon;
    muon = (Muon *) branchMuon->At(i);
    vectorOfMuons.push_back(muon);
}